In [5]:
%load_ext autoreload
%autoreload 2
from gui import SpikeImportGUI
from importers import SpikeImporter
from recordings import MNGRecording

# present the Spike GUI and let the user make his choices
spike_gui = SpikeImportGUI()
# then, create an importer object and use the gui parameters to create the recording object
importer = SpikeImporter(filepath = spike_gui.filepath, time_channel = spike_gui.time_channel, signal_channel = spike_gui.signal_channel)
recording = importer.create_recording(stimulus_channels = spike_gui.stimulus_channels, ap_channels = spike_gui.ap_channels, \
                                      force_threshold = spike_gui.force_threshold, max_ap_gap_time = spike_gui.max_ap_gap)
_ = recording.split_into_sweeps()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
List of eletrical stimuli created.
List of extra eletrical stimuli created.
Finished processing AP channel 1 out of 1
List of APs created.
Done with cropping the intervals


In [6]:
'''
Here, we extract some features from the APs:
- postpulse distance (aka latency)
- normalized signal energy
'''
%autoreload 2
from tqdm import tqdm
from feature_extraction import ResponseLatency, NormalizedSignalEnergy, SpikeCount, AdaptiveSpikeCount

print("Extracting features from every AP")
features = [ResponseLatency(regular_el_stimuli = recording.el_stimuli), \
            NormalizedSignalEnergy(), \
            SpikeCount(actpots = recording.actpots, timeframe = 100, num_intervals = 16), \
            AdaptiveSpikeCount(actpots = recording.actpots, timeframe = 100, num_splits = 8)]

for ap in tqdm(recording.actpots):
    # for each of the features, calculate and store in the ap's dict
    for feature in features:
        ap.features[feature.get_feature_name()] = feature.get_feature_value(ap)

    # also, save a reference to the previous electrical stimulus.
    # we might need that later.
    ap.prev_stimuli["regular"] = MNGRecording.get_prev_el_stimulus(ap, recording.el_stimuli)

 19%|███████████████▌                                                                | 44/227 [00:00<00:00, 431.49it/s]

Extracting features from every AP


100%|███████████████████████████████████████████████████████████████████████████████| 227/227 [00:00<00:00, 498.43it/s]


In [8]:
%autoreload 2
from plotting import FallingLeafPlot
from ipywidgets import interact_manual, fixed, IntSlider, FloatSlider

tmin, tmax = importer.get_time_range()
max_interval_length = max([stim.interval_length for stim in recording.el_stimuli])

flplot = FallingLeafPlot()
interact_manual(flplot.plot, regular_stimuli = fixed(recording.el_stimuli), action_potentials = fixed(recording.actpots), \
                t_start = FloatSlider(min = tmin, max = tmax, value = 0), \
                num_intervals = IntSlider(min = 1, max = len(recording.el_stimuli), step = 1, value = 300), \
                post_stimulus_timeframe = FloatSlider(min = 0, max = max_interval_length, step = 0.01, value = 0.05), \
                ap_tracks = fixed([]), \
                manual_name = "Update Plot")

interactive(children=(FloatSlider(value=0.0, description='t_start', max=440.0), IntSlider(value=109, descripti…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [11]:
''' 
This cell performs the clustering according to latency s.t. we can initialize a track
'''
%autoreload 2
from fibre_tracking import DBSCANClustering

# define the number of seconds that should be considered for initialization
NUM_SECONDS = 100
T_START = 0
first_track_aps = [ap for ap in recording.actpots if ap.onset > T_START and ap.onset < T_START + NUM_SECONDS]

initial_track_labels = DBSCANClustering.perform_clustering(actpots = first_track_aps, eps = 0.003, \
                                             min_samples = 10, plot_results = True)

In [19]:
%autoreload 2
from fibre_tracking.ap_track import APTrack
from plotting import FallingLeafPlot

# now, we can try to construct tracks from this
clean_track_aps = [ap for (ap, cls_idx) in zip(first_track_aps, initial_track_labels) if cls_idx == 0]
ap_track = APTrack.from_aps(sweeps = recording.sweeps, aps = clean_track_aps)

# Starting point for the falling leaf plot
FLP_START_TIME = recording.sweeps[ap_track.sweep_idcs[0]].t_start
FLP_POST_ST_TIMEFRAME = max(ap_track.latencies) * 1.5

# plot this AP track for illustrative purposes
flplot = FallingLeafPlot()
flplot.plot(regular_stimuli = recording.el_stimuli, action_potentials = recording.actpots, t_start = FLP_START_TIME, \
            num_intervals = 500, ap_tracks = [ap_track], post_stimulus_timeframe = FLP_POST_ST_TIMEFRAME, plot_raw_signal= False)

In [20]:
%autoreload 2
from fibre_tracking.ap_track import APTrack
from plotting import FallingLeafPlot

# for how many intervals do we want to extend the track?
EXT_NUM_INTV = 500
# Starting point for the falling leaf plot
FLP_START_TIME = recording.sweeps[ap_track.sweep_idcs[0]].t_start
FLP_POST_ST_TIMEFRAME = max(ap_track.latencies) * 1.5

# extend the track for the given number of intervals
ap_track.extend_downwards(recording.sweeps, num_sweeps = EXT_NUM_INTV, max_shift = 0.01, radius = 2, verbose = False)

flplot = FallingLeafPlot()
flplot.plot(regular_stimuli = recording.el_stimuli, action_potentials = recording.actpots, t_start = FLP_START_TIME, \
            num_intervals = len(ap_track), ap_tracks = [ap_track], post_stimulus_timeframe = FLP_POST_ST_TIMEFRAME, \
            plot_raw_signal= False)

The radius for median RMS calculation exceeds either the first or the last position in the sweep array. Reduce radius or increase the center sweep index to resolve this issue.


In [44]:
%autoreload 2
from fibre_tracking.ap_track import APTrack
from plotting import FallingLeafPlot

# and now, we can extract the nearest action potentials
aps = ap_track.get_nearest_existing_aps(sweeps = recording.sweeps)

# also, we may want to change the track's color :)
ap_track.color = "green"

# Starting point for the falling leaf plot
FLP_START_TIME = recording.sweeps[ap_track.sweep_idcs[0]].t_start
FLP_POST_ST_TIMEFRAME = max(ap_track.latencies) * 1.5

flplot = FallingLeafPlot()
flplot.plot(regular_stimuli = recording.el_stimuli, action_potentials = aps, t_start = FLP_START_TIME, num_intervals = len(ap_track), \
            ap_tracks = [ap_track], post_stimulus_timeframe = FLP_POST_ST_TIMEFRAME, plot_raw_signal = False)

There are no APs in sweep nr. 31
There are no APs in sweep nr. 72


In [55]:
%autoreload 2
from signal_artifacts import ActionPotential

ActionPotential.save_aps_to_csv(actpots = aps, fpath = "../data/test_ap_export.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 3501.17it/s]
